# Self Organizing Feature Map 

The goal of this notebook is to build and understand the properties of Kohonen Self-Organizing Feature Maps, an unsupervised learning technique for representing high dimensional data in much lower dimensional spaces, while maintaining the topological relationships within the training set.

We will begin first by building a model that can map colors in RGB to a 2 dimensional representation. Next, we will take the same model and apply it to a more advanced problem of learning from the MNIST dataset.



##### Importing necessary Libraries 

In [1]:
import numpy as np
import math
from PIL import Image

An SOM is a lattice made up of n by n nodes. From an initial random distribution of weights, the weights are adjusted over training to classify the input data more effectively, until it converges to a steady representation of different zones (equivalent to classes). Given an input data, the SOM can classify based as nodes in zone with similar weight vectors will be simulated.

#### Learning Algorithm 

For N Iterations:

1. Initialise weights of each node in lattice randomly
2. Select a vector from training data to compare with the lattice.
3. Choose Best Matching Unit (BMU) from lattice, by computing the most similar node to vector.
4. Compute the radius of BMU's neighbourhood. This value decreases each time-step.
5. Adjust each node within the BMU's neighbourhood to make them more similar to the input vector. Closer nodes are adusted more.
6. Repeat Step 2 to 5.

#### Defining Network 

Check out http://www.ai-junkie.com/ann/som/som3.html.

In [2]:
class SOM:
    def __init__(self, x_size, y_size, dimen, num_iter, t_step):
        # init weights to 0 < w < 256
        self.weights = np.random.randint(256, size=(x_size, y_size, dimen))\
                            .astype('float64')
        self.num_iter = num_iter
        self.map_radius = max(self.weights.shape)/2 # sigma_0
        self.t_const = self.num_iter/math.log(self.map_radius) # lambda
        self.t_step = t_step
        
    def get_bmu(self, vector):
        # calculate euclidean dist btw weight matrix and vector
        distance = np.sum((self.weights - vector) ** 2, 2)
        min_idx = distance.argmin()
        return np.unravel_index(min_idx, distance.shape)
        
    def get_bmu_dist(self, vector):
        # initialize array where values are its index
        x, y, rgb = self.weights.shape
        xi = np.arange(x).reshape(x, 1).repeat(y, 1)
        yi = np.arange(y).reshape(1, y).repeat(x, 0)
        # returns matrix of distance of each index in 2D from BMU
        return np.sum((np.dstack((xi, yi)) - np.array(self.get_bmu(vector))) ** 2, 2)

    def get_nbhood_radius(self, iter_count):
        return self.map_radius * np.exp(-iter_count/self.t_const)
        
    def teach_row(self, vector, i):
        nbhood_radius = self.get_nbhood_radius(i)
        bmu_dist = self.get_bmu_dist(vector).astype('float64')
        
        # exponential decaying learning rate
        lr = 0.1 * np.exp(-i/self.num_iter) 
        
        # influence
        theta = np.exp(-(bmu_dist)/ (2 * nbhood_radius ** 2))
        return np.expand_dims(theta, 2) * (vector - self.weights)
        
    def teach(self, t_set):
        for i in range(self.num_iter):
            if i % 10 == 0:
                print("Training Iteration: ", i)
            for j in range(len(t_set)):
                self.weights += self.teach_row(t_set[j], i)
        
    def show(self):
        im = Image.fromarray(self.weights.astype('uint8'), mode='RGB')
        im.format = 'JPG'
        im.show()

#### Training the network 

In [6]:
# Hyperparameters
map_w = 200
map_h = 200
data_dim = 3
epochs = 100
t_step = 1

# Initialize random RGB dataset
train_set = np.random.randint(256, size=(15,3))

# Defining map
s = SOM(map_w, map_h, data_dim, epochs, t_step)

# Start Training
s.teach(train_set)
s.show()

Training Iteration:  0
Training Iteration:  10
Training Iteration:  20
Training Iteration:  30
Training Iteration:  40
Training Iteration:  50
Training Iteration:  60
Training Iteration:  70
Training Iteration:  80
Training Iteration:  90


#### Training on MNIST Dataset 

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [8]:
# Using first 10000 images for training
train_data = mnist.train.images[:10000,:]

# Converting normalized data to values between 0 and 256
train_data = train_data * 256

In [9]:
map_w = 20
map_h = 20
data_dim = 784
epochs = 100
t_step = 1

# Defining map
mnist_map = SOM(map_w, map_h, data_dim,epochs, t_step)

#Start training
mnist_map.teach(train_data)

# Converting 3D SOM to 2D image
map_matrix = np.zeros((560,560))
for i in range(map_w):
    for j in range(map_h):
        # Reshaping 768 weight vector to 28x28 matrix
        reshaped_weights = mnist_map.weights[i][j].reshape((28, 28))
        # Assigning matrix to respective position of node in lattice
        map_matrix[i*28:i*28+28, j*28:((j*28)+28)] = reshaped_weights

Training Iteration:  0
Training Iteration:  10
Training Iteration:  20
Training Iteration:  30
Training Iteration:  40
Training Iteration:  50
Training Iteration:  60
Training Iteration:  70
Training Iteration:  80
Training Iteration:  90


In [11]:
# Showing Image
map_img = Image.fromarray(map_matrix.astype('uint8'))
map_img.format = 'JPG'
map_img.show()

The output looks like this:

<img src = "mnistpic.jpeg">